# ALL LACs of ALL MSSs

## Import Modules

In [1]:
# Import modules (NOK_ERI_Toolkit) & others that will be needed along the program
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import NOK_ERI_Toolkit as NOK_ERI_Toolkit

%matplotlib inline
%load_ext autoreload
%autoreload 2


## Main Program

### Initialize variables

In [2]:
# Initialize NOKIA, ERICSSON & COMMON variables

# NOKIA
NOK_MSS_Name = '0'
NOK_MSS_Date = '0'
NOK_MSS_Time = '0'
RNC_NAME = '0'
RNC_MCC = '0'
RNC_MNC = '0'
RNC_ID = '0'
RNC_MUL_PLMN = '0'
RNC_STAT = '0'
RNC_OSTAT = '0'
RNC_LAC_MCC = '0'
RNC_LAC_MNC = '0'
RNC_LAC_NO = '0'

NOK_MSS_Pars = [NOK_MSS_Name, NOK_MSS_Date, NOK_MSS_Time]
RNC_Pars = [RNC_NAME, RNC_MCC, RNC_MNC, RNC_ID, RNC_MUL_PLMN, RNC_STAT, RNC_OSTAT]
RNC_LAC_Pars = [RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

NOK_index = 0
NOK_position = 0
NOK_RNC_LAC_count = 0
NOK_RNC_LAC_flag = 0

In [3]:
# Create & Initialize & Set the NOKIA & ERICSSON pd.DataFrame

# NOKIA
col_names = ['MSS','Date','Time','RNC','RNC_MCC','RNC_MNC','RNC_ID','RNC_MULT_PLMN',
                'RNC_STAT','RNC_OP_STAT','MCC','MNC','LAC']
df_MSS_RNC_LACs = pd.DataFrame(columns=col_names)
df_MSS_RNC_LACs.loc[0] = [NOK_MSS_Name, NOK_MSS_Date, NOK_MSS_Time, RNC_NAME,
                            RNC_MCC, RNC_MNC, RNC_ID, RNC_MUL_PLMN, RNC_STAT, RNC_OSTAT,
                            RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

In [4]:
# Especify the path of the NOKIA & ERICSSON log files 
NOK_path_to_file = 'C:/Users/Alvaro/OneDrive/My Files/_Scripts_Models/Commands Generators Logs/20220204 - ZE2I - All_MSS.txt'

### Load log file

In [5]:
# Load the log file
NOK_log_file = NOK_ERI_Toolkit.open_file(NOK_path_to_file)

### Execute NOKIA PROGRAM

In [6]:
# Run the main program (txt => pd.DataFrame)
while (NOK_index < len(NOK_log_file)-1):
    NOK_index, NOK_position, NOK_MSS_Pars = NOK_ERI_Toolkit.get_mss_pars(NOK_log_file, NOK_index,'ZE2I::RT=ALL;')
    while (NOK_index < len(NOK_log_file)-1) and (not NOK_position == 999):
        NOK_index, NOK_position, RNC_Pars = NOK_ERI_Toolkit.get_ze2i_rnc_pars(NOK_log_file, NOK_index)
        while (NOK_index < len(NOK_log_file)-1) and (not NOK_position == 999):
            NOK_index, NOK_position, NOK_RNC_LAC_flag, RNC_LAC_Pars = NOK_ERI_Toolkit.get_ze2i_rnc_lac_pars(NOK_log_file, NOK_index, NOK_RNC_LAC_flag)
            if NOK_position == 999:                                                 # if 'COMMAND EXECUTED'
                NOK_index = NOK_index + 1                                           # Increase index to read next line
                break
            else:
                if (NOK_RNC_LAC_flag == 1):
                    new_RNC_LAC = NOK_MSS_Pars + RNC_Pars + RNC_LAC_Pars
                    df_MSS_RNC_LACs.loc[NOK_RNC_LAC_count] = new_RNC_LAC
                    NOK_RNC_LAC_count = NOK_RNC_LAC_count + 1
                else:
                    break

### Drop columns & prepare to merge

In [7]:
# NOKIA DataFrame => Drop some columns to get just the most important ones.

df_MSS_RNC_LAIs = df_MSS_RNC_LACs
df_MSS_RNC_LAIs['MSS'] = df_MSS_RNC_LAIs['MSS'].str.replace(r'0', '')
#df_MSS_RNC_LAIs['RNC_ID'] = df_MSS_RNC_LAIs['RNC_ID'].str.replace(r'0', '')
df_MSS_RNC_LAIs['LAI'] =  df_MSS_RNC_LAIs['MCC'] + '-' + df_MSS_RNC_LAIs['MNC'] + '-' + df_MSS_RNC_LAIs['LAC']
df_MSS_RNC_LAIs['MSS-LAI'] =  df_MSS_RNC_LAIs['MSS'] + '-' + df_MSS_RNC_LAIs['LAI']
df_MSS_RNC_LAIs['NE-LAI'] =  df_MSS_RNC_LAIs['RNC'] + '-' + df_MSS_RNC_LAIs['LAI']

NOK_drop_col = ['RNC_MCC', 'RNC_MNC', 'RNC_MULT_PLMN', 'RNC_STAT', 'RNC_OP_STAT', 'RNC_ID']
df_MSS_RNC_LAIs = df_MSS_RNC_LAIs.drop(columns=NOK_drop_col)
NOK_col = ['MSS', 'Date', 'Time', 'MCC', 'MNC', 'LAC', 'LAI', 'MSS-LAI', 'NE-LAI', 'RNC']
df_MSS_RNC_LAIs = df_MSS_RNC_LAIs[NOK_col]
df_MSS_RNC_LAIs

,MSS,Date,Time,MCC,MNC,LAC,LAI,MSS-LAI,NE-LAI,RNC
0,MSSBA6,2022-02-04,09:47:18,724,05,30971,724-05-30971,MSSBA6-724-05-30971,RNCBA05-724-05-30971,RNCBA05
1,MSSBA6,2022-02-04,09:47:18,724,05,31171,724-05-31171,MSSBA6-724-05-31171,RNCBA05-724-05-31171,RNCBA05
2,MSSBA6,2022-02-04,09:47:18,724,05,31571,724-05-31571,MSSBA6-724-05-31571,RNCBA05-724-05-31571,RNCBA05
3,MSSBA6,2022-02-04,09:47:18,724,05,31071,724-05-31071,MSSBA6-724-05-31071,RNCBA05-724-05-31071,RNCBA05
4,MSSBA6,2022-02-04,09:47:18,724,05,31671,724-05-31671,MSSBA6-724-05-31671,RNCBA05-724-05-31671,RNCBA05
...,...,...,...,...,...,...,...,...,...,...
218,MSSBAA,2022-02-04,09:46:14,724,28,33675,724-28-33675,MSSBAA-724-28-33675,RNCTCSB-724-28-33675,RNCTCSB
219,MSSBAA,2022-02-04,09:46:14,724,05,33975,724-05-33975,MSSBAA-724-05-33975,RNCTCSB-724-05-33975,RNCTCSB
220,MSSBAA,2022-02-04,09:46:14,724,28,33975,724-28-33975,MSSBAA-724-28-33975,RNCTCSB-724-28-33975,RNCTCSB
221,MSSBAA,2022-02-04,09:46:14,724,05,34075,724-05-34075,MSSBAA-724-05-34075,RNCTCSB-724-05-34075,RNCTCSB


### Plot + Insights

In [8]:
# Continue...

In [9]:
df_MSS_LACs.head()

NameError: name 'df_MSS_LACs' is not defined

In [ ]:
df_MSS_LACs.groupby('MSS')['AT'].count()

MSS
MSSBA06     47
MSSBA09    181
MSSBA0A    207
Name: AT, dtype: int64

In [ ]:
df_MSS_LACs.describe(include='all')

,MSS,Date,Time,MCC,MNC,LAC_str,LAC_NAME,AT,INT,RNGP,MNC_AL,DSAV,TZ,HONLA
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,3,1,3,1,3,352,426,1,1,9,2,1,1,9
top,MSSBA0A,2022-02-04,09:50:20,724,05,40174,LAC1577,001,0600 ( x 10 msec ),005,-,-,-,005
freq,207,435,207,435,346,2,2,435,435,148,434,435,435,148


In [ ]:
df_MSS_LACs['MSS'].describe(include='all')

count         435
unique          3
top       MSSBA0A
freq          207
Name: MSS, dtype: object

In [ ]:
df_MSS_LACs.head()

,MSS,Date,Time,MCC,MNC,LAC_str,LAC_NAME,AT,INT,RNGP,MNC_AL,DSAV,TZ,HONLA
0,MSSBA06,2022-02-04,09:51:20,724,05,01671,LAC01671,001,0600 ( x 10 msec ),001,-,-,-,001
1,MSSBA06,2022-02-04,09:51:20,724,05,02575,LAC2575,001,0600 ( x 10 msec ),002,-,-,-,002
2,MSSBA06,2022-02-04,09:51:20,724,05,02775,LAC2775,001,0600 ( x 10 msec ),002,-,-,-,002
3,MSSBA06,2022-02-04,09:51:20,724,05,01575,LAC1575,001,0600 ( x 10 msec ),002,-,-,-,002
4,MSSBA06,2022-02-04,09:51:20,724,05,01475,LAC1475,001,0600 ( x 10 msec ),002,-,-,-,002


In [ ]:
df_MSS_LACs['AT'].count()

435

In [ ]:
df_MSS_LACs

,MSS,Date,Time,MCC,MNC,LAC_str,LAC_NAME,AT,INT,RNGP,MNC_AL,DSAV,TZ,HONLA
0,MSSBA06,2022-02-04,09:51:20,724,05,01671,LAC01671,001,0600 ( x 10 msec ),001,-,-,-,001
1,MSSBA06,2022-02-04,09:51:20,724,05,02575,LAC2575,001,0600 ( x 10 msec ),002,-,-,-,002
2,MSSBA06,2022-02-04,09:51:20,724,05,02775,LAC2775,001,0600 ( x 10 msec ),002,-,-,-,002
3,MSSBA06,2022-02-04,09:51:20,724,05,01575,LAC1575,001,0600 ( x 10 msec ),002,-,-,-,002
4,MSSBA06,2022-02-04,09:51:20,724,05,01475,LAC1475,001,0600 ( x 10 msec ),002,-,-,-,002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,MSSBA0A,2022-02-04,09:50:20,724,05,52079,LAC52079,001,0600 ( x 10 msec ),007,-,-,-,007
431,MSSBA0A,2022-02-04,09:50:20,724,05,23875,LAC23875,001,0600 ( x 10 msec ),004,-,-,-,004
432,MSSBA0A,2022-02-04,09:50:20,724,28,40675,LAC40675RS,001,0600 ( x 10 msec ),005,-,-,-,005
433,MSSBA0A,2022-02-04,09:50:20,724,05,40675,LAC40675,001,0600 ( x 10 msec ),005,-,-,-,005


In [ ]:
df_MSS_LACs.dropna().describe()

,MSS,Date,Time,MCC,MNC,LAC_str,LAC_NAME,AT,INT,RNGP,MNC_AL,DSAV,TZ,HONLA
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,3,1,3,1,3,352,426,1,1,9,2,1,1,9
top,MSSBA0A,2022-02-04,09:50:20,724,05,40174,LAC1577,001,0600 ( x 10 msec ),005,-,-,-,005
freq,207,435,207,435,346,2,2,435,435,148,434,435,435,148


In [ ]:
df_MSS_LACs.groupby('MSS', axis=1).count()

ValueError: operands could not be broadcast together with shapes (14,) (435,13) 